<a href="https://colab.research.google.com/github/chrishuson/mathai/blob/master/Colab_roster_home.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# Versions
# pd.show_versions(as_json=False)
os.getcwd()

'/content'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
os.listdir('/content/drive/My Drive/Class_lists')

['HS Master Contact List: 2020 2021.gsheet',
 'HS Rosters.gsheet',
 'HS-roster-Oct10.csv',
 '1-5Deltamath-.xlsx',
 '1-5Deltamath.xlsx',
 'scores.csv',
 'HS_Master_Contact_List_2020_2021_HS_List.csv',
 'HS_Rosters_Master_List_Oct12.csv',
 'No_Deltamath.csv',
 'scores_deltamath.csv',
 'scores_deltamath.gsheet']

In [6]:
# Import and format HS and 10th grade roster dataframes
HS_roster_df = pd.read_csv('/content/drive/My Drive/Class_lists/HS_Rosters_Master_List_Oct12.csv')
HS_roster_df.dropna(subset=['Student'], inplace=True)

# No comma in Student name
HS_roster_df.loc[~HS_roster_df.Student.str.contains(','), 'Student'] = HS_roster_df[~HS_roster_df.Student.str.contains(',')].Student.str.replace(' ', ', ')
HS_roster_df[HS_roster_df.OSIS.isnull()]

,Student,OSIS,Learning Model,Advisor,Group,Schedule,BECA324 Email


In [7]:

HS_roster_df.OSIS = HS_roster_df.OSIS.apply(int)
new_column_names = {'Learning Model':'Learning_model', 'BECA324 Email':'Email'}
HS_roster_df.rename(columns=new_column_names, inplace=True)

HS_roster_df.Student = HS_roster_df.Student.str.replace('-', ' ').str.title()
#HS_roster_df.Student = HS_roster_df.Student.str.title()
HS_roster_df[['Last', 'First']] = HS_roster_df.Student.str.split(',', expand=True)
HS_roster_df.First = HS_roster_df.First.str.strip()
HS_roster_df.Last = HS_roster_df.Last.str.strip()
HS_roster_df.Last = HS_roster_df.Last.str.split(' ').apply(lambda x: x[0]) # causes a problem for "De Los Santos", and similar

HS_roster_df['First'] = HS_roster_df['First'].replace(['Nathanial Richard', 'Bianca Anelis', 'Ethan Allen', 'Jaylin Noel', 'John Paul', 'Yiselle', 'Hilary'],
                                                    ['Nathanial', 'Bianca', 'Ethan', 'Jaylin', 'John Paul', 'Yisselle', 'Hillary'])

cols_to_order = ['First', 'Last', 'Schedule', 'Advisor', 'Email']
new_columns = cols_to_order + (HS_roster_df.columns.drop(cols_to_order).tolist())
HS_roster_df = HS_roster_df[new_columns]
HS_roster_df = HS_roster_df.sort_values(['Schedule', 'First', 'Last'])

HS_roster_df[HS_roster_df.Schedule.str.startswith('10')].head()

,First,Last,Schedule,Advisor,Email,Student,OSIS,Learning_model,Group
271,Aiden,Sandoval,10B,Krzeminski,asandoval1@beca324.org,"Sandoval, Aiden",221038003,Hybrid,10B
248,Carlos,Rodriguez,10B,Easton,crodriguez1@beca324.org,"Rodriguez, Carlos",216045260,Hybrid,10B
111,Juliet,Gonzalez,10B,Maldonado,jgonzalez1@beca324.org,"Gonzalez, Juliet",224956979,Hybrid,10B
163,Tyquan,Mcclary,10B,Spaeth,tmcclary1@beca324.org,"Mcclary, Tyquan",220772693,Hybrid,10B
237,Bianca,Reyes,10C,Huson,breyes1@beca324.org,"Reyes, Bianca Anelis",223011388,Hybrid,10C


In [ ]:
HS_summary = HS_roster_df.value_counts('Schedule').reset_index()
HS_summary.columns = ['Schedule', 'Cohort_size']
HS_summary['Grade'] = HS_summary.Schedule.str[0:2]
HS_summary['Section'] = HS_summary.Schedule.str[2:]
HS_summary.pivot_table(values='Cohort_size', index='Grade', columns='Section', fill_value='')


In [9]:
roster10_df = HS_roster_df[HS_roster_df.Schedule.str.startswith('10')].copy().reset_index(drop=True)
print(roster10_df.columns)
print('HS len: ', len(HS_roster_df), '\n10th len: ', len(roster10_df))
roster10_df.value_counts('Schedule')

Index(['First', 'Last', 'Schedule', 'Advisor', 'Email', 'Student', 'OSIS',
       'Learning_model', 'Group'],
      dtype='object')
HS len:  325 
10th len:  90


Schedule
10D2    43
10D1    37
10C      6
10B      4
dtype: int64

In [ ]:
classroom_raw_df = pd.read_csv('/content/drive/My Drive/Class_lists/Geometry_10_Dr_Huson.csv')
classroom_raw_df.columns

In [35]:
classroom_df =pd.DataFrame()
scored_columns = ['Last Name', 'First Name', 'Unit 1 Attendance', '1.5 Deltamath Algebra and Graphing practice',
                  'Homework: watch first Geometry unit of Khan Academy', '1.5 Worksheet: Notation and terminology, skills',
                  '1.7 Construction of an equilateral triangle','1.7 Genius Scan app on your phone to snap photos and upload assignments', 'Email Address',]
classroom_df[['Last', 'First', 'Attendance', 'Deltamath', 'Khan', 'Worksheet', 'Construction', 'Genius', 'Email']] = classroom_raw_df[scored_columns]
classroom_df.head()

,Last,First,Attendance,Deltamath,Khan,Worksheet,Construction,Genius,Email
0,Sandoval,(10B)Aiden,3,3,1,3,3,1,asandoval1@beca324.org
1,Rodriguez,(10B)Carlos,3,1,1,1,3,1,crodriguez1@beca324.org
2,Mcclary,(10B)Tyquan,3,1,1,1,3,1,tmcclary1@beca324.org
3,Reyes,(10C)Bianca,3,2,3,1,3,1,breyes1@beca324.org
4,Delgado,(10C)Donovan,3,4,4,3,3,3,ddelgado1@beca324.org


In [48]:
g_scores_df = roster10_df.merge(classroom_df, how='inner', on='Email', suffixes=(None, '_y'))
print('Google Classroom roster length: ', len(classroom_df))
print('10th Grade Master length: ', len(roster10_df))
print('Merged roster length: ', len(g_scores_df))
#g_scores_df[g_scores_df.Last.str[:4] != g_scores_df.Last_y.str.title().str[:4]]
g_scores_df.drop(columns=['Student', 'OSIS', 'Learning_model', 'Group', 'Last_y', 'First_y'])

Google Classroom roster length:  86
10th Grade Master length:  90
Merged roster length:  86


,First,Last,Schedule,Advisor,Email,Attendance,Deltamath,Khan,Worksheet,Construction,Genius
0,Aiden,Sandoval,10B,Krzeminski,asandoval1@beca324.org,3,3,1,3,3,1
1,Carlos,Rodriguez,10B,Easton,crodriguez1@beca324.org,3,1,1,1,3,1
2,Tyquan,Mcclary,10B,Spaeth,tmcclary1@beca324.org,3,1,1,1,3,1
3,Bianca,Reyes,10C,Huson,breyes1@beca324.org,3,2,3,1,3,1
4,Donovan,Delgado,10C,Johnsen,ddelgado1@beca324.org,3,4,4,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...
81,Steven,Lopez,10D2,Spaeth,slopez2@beca324.org,1,1,3,1,1,1
82,Tigidanke,Dukuray,10D2,Krzeminski,tdukuray1@beca324.org,2,2,3,1,3,3
83,Yisselle,Perez,10D2,Maldonado,yperezsantiago1@beca324.org,2,2,2,1,3,1
84,Yolanda,Tello,10D2,O'Grady,ytello1@beca324.org,3,3,3,1,2,3


In [ ]:
deltamath_df = pd.read_excel('/content/drive/My Drive/Class_lists/1-5Deltamath.xlsx', skipfooter=6)
deltamath_df.drop(columns='Class', inplace=True)
deltamath_df.columns = ['First', 'Last', 'Deltamath_1_5', 'Deltamath_1_4', 'Deltamath_1_2']
deltamath_df[['First', 'Last']] = deltamath_df[['First', 'Last']].apply(lambda x: x.str.strip())
deltamath_df.Last = deltamath_df.Last.str.split(' ').apply(lambda x: x[0])

print('Deltamath accounts: ', len(deltamath_df))
deltamath_df.tail(45)

In [50]:
deltamath_df.First = deltamath_df.First.replace('Lena', 'Leanne')
#deltamath_df[deltamath_df.First.str.contains('Le')]

In [ ]:
scores_df = roster10_df.merge(deltamath_df, how='outer').sort_values(['Schedule', 'First', 'Last'])
scores_df[scores_df[['Schedule', 'Deltamath_1_5']].isnull().any(axis=1)] # Find unmatched

In [52]:
scores_df.fillna(0, inplace=True)

In [ ]:
scores_df[['First', 'Last', 'Deltamath_1_5', 'Deltamath_1_4', 'Deltamath_1_2']].boxplot()

In [53]:
def rubric(d):
  total = sum(d)
  if total ==  300:
    return 4
  elif total > 200:
    return 3
  elif total >= 75:
    return 2
  else:
    return 1

In [ ]:
#scores_df[['Deltamath_1_5', 'Deltamath_1_4', 'Deltamath_1_2']].apply(lambda x: 4 if x[0] else x[1], axis=1)
scores_df['Deltamath'] = scores_df[['Deltamath_1_5', 'Deltamath_1_4', 'Deltamath_1_2']].apply(rubric, axis=1)
scores_df[['First', 'Last', 'Deltamath', 'Deltamath_1_5', 'Deltamath_1_4', 'Deltamath_1_2']].sort_values(['Deltamath', 'Deltamath_1_5']).iloc[20:60]

In [65]:
combined_scores_df = g_scores_df.merge(scores_df, how='left', on=['Last', 'First'], suffixes=(None, '_raw'))
combined_scores_df[combined_scores_df.Deltamath != combined_scores_df.Deltamath_raw][['First', 'Last', 'Deltamath', 'Deltamath_raw', 'Deltamath_1_5', 'Deltamath_1_4', 'Deltamath_1_2']]

,First,Last,Deltamath,Deltamath_raw,Deltamath_1_5,Deltamath_1_4,Deltamath_1_2


In [76]:
scores_df[scores_df.Deltamath_1_5.isnull()][['First', 'Last', 'Schedule', 'Advisor', 'Email']].to_csv('/content/drive/My Drive/Class_lists/No_Deltamath.csv')

In [66]:
combined_scores_df.columns

Index(['First', 'Last', 'Schedule', 'Advisor', 'Email', 'Student', 'OSIS',
       'Learning_model', 'Group', 'Last_y', 'First_y', 'Attendance',
       'Deltamath', 'Khan', 'Worksheet', 'Construction', 'Genius',
       'Schedule_raw', 'Advisor_raw', 'Email_raw', 'Student_raw', 'OSIS_raw',
       'Learning_model_raw', 'Group_raw', 'Deltamath_1_5', 'Deltamath_1_4',
       'Deltamath_1_2', 'Deltamath_raw'],
      dtype='object')

In [68]:
combined_scores_df[['Schedule', 'First', 'Last', 'Attendance', 'Khan', 'Worksheet', 'Construction', 
                    'Genius', 'Deltamath', 'Deltamath_1_5',
       'Deltamath_1_4', 'Deltamath_1_2', 'Advisor', 'Email']]\
       .sort_values(['Schedule', 'First', 'Last'])\
       .to_csv('/content/drive/My Drive/Class_lists/combined_scores.csv')

In [19]:
scores_df.to_csv('/content/drive/My Drive/Class_lists/scores.csv')

In [ ]:
scores_df = pd.read_csv('/content/drive/My Drive/Class_lists/scores.csv')
scores_df

In [9]:
scores_df[scores_df.Last == 'Arvelo']

,Unnamed: 0,Cohort,First,Last,Deltamath_1_5,Deltamath_1_4,Deltamath_1_2,Advisor,Email,IEP,ENL,Worksheet_1_5
35,35,10D1,Natalie,Arvelo,0.0,75.0,100.0,Johnsen,narvelo1@beca324.org,ICT,Expanding,0


In [8]:
scores_df['Worksheet_1_5'] = 0

In [13]:
worksheet_credit = ['Sandoval', 'Delgado', 'Arvelo', 'Rosario', 'Mosso', 'Frimpong', 'Masoud']
scores_df.loc[scores_df.Last.isin(worksheet_credit), 'Worksheet_1_5'] = 100

In [17]:
scores_df.loc[scores_df.First.isin(['Norma', 'Alissa', 'Leanne']), 'Worksheet_1_5'] = 100